[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/B1AAB/GraphStudio/blob/main/g101/g101.ipynb)

# Get to Know Bitcoin Graph

The Bitcoin Graph dataset and its accompanying suite of tools and resources are purpose-built to bridge the cryptocurrency and ML communities. It is discussed in detail in the following paper:

> Vahid Jalili. (2025). **The Temporal Graph of Bitcoin Transactions.** _In Advances in Neural Information Processing Systems (NeurIPS)._
>
> https://arxiv.org/abs/2510.20028


Briefly, the suite contains:

- ETL pipeline, [EBA](https://github.com/B1AAB/EBA):

  - Interfaces with the Bitcoin network and parses every transaction recorded on the blockchain.

  - Encodes transactions as graphs, yielding batched sets of nodes and edges in TSV files.

  - Imports the batches into a Neo4j database for scalable querying of deep transactional context and entity relationships.

  - Implements specialized sampling algorithms that generate sub-graphs for training graph neural network models.

- [Graph Studio](https://github.com/B1AAB/GraphStudio):

  - Showcases a suite of impactful community-developed applications on the graph.


These resources are organized by application scenarios.

This notebook serves as a guided tour of the [Bitcoin Graph](https://registry.opendata.aws/eba) dataset, and contains the following sections.


* The _Seeds_ 🌱 **Granular Graph Data**:

  🏃 skim through raw data.


* The _Trunk & Roots_ 🌳 **The Full Graph Database**:

  🏃 skim through loading data into a Neo4j database.

* The _Branches_ 🌿 **Sampled Communities**:

  🏃 skim through community sampling and pre-sampled _hello-world_ communities.

* The _Fruits_ 🍎 **Building Models**:

  🏃 skim through model training, and
  
  🚶 focused model evaluation.


# Setup

#### Install & Import Dependencies

In [ ]:
!pip install boto3>=1.38.23
!mkdir -p /tmp/b1aab/graphstudio
!git clone https://github.com/B1AAB/GraphStudio /tmp/b1aab/graphstudio

!pip install -r /tmp/b1aab/graphstudio/quickstart/script_classification/requirements.txt
!pip install -e /tmp/b1aab/graphstudio/quickstart/script_classification
!pip install kagglehub
!pip install gdown

In [ ]:
import os
import sys
pkg_path = '/tmp/b1aab/graphstudio/quickstart/script_classification'
if pkg_path not in sys.path:
    sys.path.insert(0, pkg_path)

In [ ]:
import torch
import kagglehub
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib.ticker import FuncFormatter
import numpy as np
import json
import gdown
import tempfile
import networkx as nx
import zipfile
import io

import boto3
from botocore import UNSIGNED
from botocore.config import Config

from script_classification.data_loader import BitcoinScriptsDataset
from script_classification.models import GraphEncoder
from script_classification.evaluation.embeddings import embed_roots
from script_classification.evaluation.clustering import find_optimal_clusters_root, plot_find_optimal_clusters_root

In [ ]:
working_dir = tempfile.gettempdir()

#### Configure Environment

In [ ]:
sns.set_theme()

In [ ]:
SEED = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# The _Seeds_ 🌱 **Granular Graph Data**

The Bitcoin graph dataset provides a complete history of transactions recorded on the Bitcoin blockchain, specifically structured for machine learning. The [v1 release](https://eba.b1aab.ai/releases/data-release/v1) release consists of over `2.4` billion nodes and `39.72` billion edges.


In this release, the nodes and edges are provided in batched `.tsv.gz` files. Each batch contains nodes and edges representing transactions from a contiguous set of Bitcoin blocks. The batches are organized to have a relatively similar number of nodes and edges; consequently, the number of blocks covered by each batch varies.

Each batch is structured as follows:

* It contains one file for each node type and one file for each edge type.

* File names are prefixed with a batch ID (the Unix timestamp when the ETL pipeline processed the batch) and the node or edge type, and they end with `.tsv.gz`.


Additionally, since the dataset is prepared for easier import into a Neo4j database, the TSV files in each batch do not contain headers. Instead, there is a separate file for each node and edge type, without a batch prefix, that contains headers only.

Moreover, there are three files named `0_BitcoinGraph.tsv.gz`, `unique_BitcoinScriptNode.tsv.gz`, and `unique_BitcoinTxNode.tsv.gz` that contain all unique block, script, and transaction nodes, respectively, across all batches.

## Dataset overview

In the following we list versions of the dataset hosted on AWS Open Data.

In [ ]:
bucket="bitcoin-graph"
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

In [ ]:
for item in s3.list_objects_v2(Bucket=bucket, Delimiter='/')['CommonPrefixes']:
  print(f"Dataset versions: {item['Prefix']}")

Objects count in the v1 of the dataset:

In [ ]:
s3_resource = boto3.resource("s3", config=Config(signature_version=UNSIGNED))
bucket_obj = s3_resource.Bucket(bucket)
obj_count = sum(1 for _ in bucket_obj.objects.filter(Prefix="v1/data_to_import_neo4j/"))
print(f"Objects count: {obj_count:,}")

Nodes and edges in one example batch.

In [ ]:
for item in s3.list_objects_v2(Bucket=bucket, Prefix="v1/data_to_import_neo4j/1737505816_", MaxKeys=100)["Contents"]:
  print(item["Key"])

Next, we download the file containing `script-to-script` edges and its corresponding header file, then create a pandas dataframe using this data.

In [ ]:
with s3.get_object(Bucket=bucket, Key="v1/data_to_import_neo4j/1737505816_BitcoinS2S.tsv.gz")["Body"] as file_object:
  df = pd.read_csv(file_object, compression="gzip", sep="\t")

with s3.get_object(Bucket=bucket, Key="v1/data_to_import_neo4j/BitcoinS2S_header.tsv.gz")["Body"] as header_object:
  header_series = pd.read_csv(header_object, compression="gzip", sep="\t", header=None).iloc[0]
df.columns = header_series.tolist()

In [ ]:
df.head()

The headers have been adjusted for importing data into a Neo4j database. Each row in the data represents a `script-to-script` edge.

* `:START_ID(Script)`: Specifies the ID of the start node and identifies its label as `Script`.

* `:END_ID(Script)`: Specifies the ID of the end node and identifies its label as `Script`.

* `Value:float`: The transaction amount in BTC. (The `:float` suffix is a type hint for Neo4j; it can be ignored in the dataframe.)`

* `Height:int`: The block height of the transaction. (The `:int` suffix is a type hint for Neo4j and can be ignored in the dataframe.)`

* `:TYPE`: Sets the edge type in Neo4j.  


## Getting Started with Graph Analysis

You can run graph analysis algorithms on this dataset, perhaps by focusing on a time span that's most interesting to your application (i.e., a subset of batches). Since node and edge types are in separate files, you can use a single source (e.g., `script-to-script`) or try combining multiple edge and node types, depending on your application.

You can also use this dataset for training graph neural network models. Additionally, since you have `Value` and `Height`, you could build models to predict transfers (like forecasting the number and volume of transfers in future blocks). You could even combine this data with market data to create price prediction models.

For demonstration, below we explore some simple summary statistics from the graph using the `networkx` library.

In [ ]:
# Rename columns to be more intuitive
df = df.rename(columns={
    ":START_ID(Script)": "source",
    ":END_ID(Script)": "target",
    "Value:float": "value",
    "Height:int": "height",
    ":TYPE": "type"
})

In [ ]:
# Create a directed graph
G = nx.from_pandas_edgelist(df, source="source", target="target", edge_attr=True, create_using=nx.DiGraph)

In [ ]:
print(f"   Nodes count (script): {G.number_of_nodes():,}")
print(f"Edges count (transfers): {G.number_of_edges():,}")
print(f"          Graph density: {nx.density(G):.6f}")

In [ ]:
# Check nodes withest out-degree
out_degrees = G.out_degree()
sorted_out_degrees = sorted(out_degrees, key=lambda item: item[1], reverse=True)

for i, (node, degree) in enumerate(sorted_out_degrees[:3]):
    print(f"{i+1}. Out-degree: {degree:,}\tScript node: {node}")

## Getting Started: Ideas and Directions

This raw data already empowers you to build many impactful applications, and because it's batched, you can opt to use only the data relevant to your needs. However, you can develop even more impactful applications by leveraging the deeper insights obtained from loading this data into a graph database, which we cover in the next section.




# The _Trunk & Roots_ 🌳 **The Full Graph Database**

One of the greatest potentials of the Bitcoin graph is its ability to model long-tail longitudinal trading and analyze node neighborhoods many hops away. This combination of temporal and structural data provides deep context for the transactions and behaviors recorded on the blockchain. Leveraging this potential on a multi-billion node graph, however, requires highly efficient retrieval methods.

To deliver on this, we have loaded the batches of nodes and edges (discussed above) into a Neo4j database. We can now leverage Neo4j's capabilities for efficient, deep-context querying of the graph.

Importing these batches is a highly computationally intensive process that can take a considerable amount of time (see the [import guide](https://eba.b1aab.ai/docs/bitcoin/etl/import) and [scalability challenges](https://eba.b1aab.ai/docs/gs/accessibility)). To make this resource widely accessible, we provide the database dump. This allows you to skip the bulk import step and simply restore the prepared dataset in Neo4j, which is significantly faster and requires fewer computational resources.

In [ ]:
bucket="bitcoin-graph"
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

The `neo4j_db_dump/` bucket contains neo4j database dump.

Count of objects in this bucket:

In [ ]:
s3_resource = boto3.resource("s3", config=Config(signature_version=UNSIGNED))
bucket_obj = s3_resource.Bucket(bucket)
obj_count = sum(1 for _ in bucket_obj.objects.filter(Prefix="v1/neo4j_db_dump/"))
print(f"Objects count: {obj_count:,}")

The database dump is provided as a multi-part `.gz` archive, split into 700 MB partitions. We list a few of the files in this bucket below. You can find instructions on how to load the database into Neo4j on [this documentation page](https://eba.b1aab.ai/docs/bitcoin/etl/import#load).

In [ ]:
for item in s3.list_objects_v2(Bucket=bucket, Prefix="v1/neo4j_db_dump/neo4j.dump", MaxKeys=10)["Contents"]:
  print(item["Key"])

Running the full dataset in Neo4j is highly resource-intensive (requiring ~4TB of storage, [details](https://eba.b1aab.ai/docs/bitcoin/etl/overview)), making it beyond the scope of a _101_ tutorial and impossible to run in a notebook.

Therefore, in the next section, we'll work with sampled communities to demonstrate the database's potential. This approach allows you to explore the data's value before committing to the resources needed to run a full database instance yourself.

# The _Branches_ 🌿 **Sampled Communities**

A common practice for developing machine learning models on such a large graph is to work with sampled communities. However, given that the Bitcoin Graph covers over 16 years of real-world financial trades where trading patterns and volume have significantly evolved, sampling must be application-focused to generate communities that support a specific training objective.

As discussed, sampling your own application-specific communities requires a running Neo4j database. You can then follow the steps in [this documentation]((https://eba.b1aab.ai/docs/bitcoin/sampling/overview)) to use the tools EBA provides for this purpose.

However, for a quick start that doesn't require hosting a Neo4j database, we have also provided generically sampled communities. We will now go through the steps for accessing them.

In [ ]:
bucket="bitcoin-graph"
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

for item in s3.list_objects_v2(Bucket=bucket, Prefix="v1/sampled_communities/", MaxKeys=10)["Contents"]:
  print(item["Key"])

The `v1/sampled_communities/script_to_script_200k.zip` archive contains `200 000` communities sampled with the following configuration:

* Communities count: `200 000`
* Node and edge type: `script-to-script` edges of `transfer` and `fee` types.
* Sampling algorithm: _Forest-Fire_ sampling, with the following parameters:
  * Node count per community: `[25, 1000]`
  * Edge count per community: `[24, 10000]`
  * Node count at root: `25`
  * Max hops: `3`
  * Node count reduction factor by hop: `5`

(For a detailed description of these parameters, you may refer to the [paper](https://arxiv.org/abs/2510.20028) or the [method documentation](https://eba.b1aab.ai/docs/bitcoin/sampling/forest-fire).)

Next, we download and unzip the archive.

Note, to simplify accessing the sample graphs to the ML community, we are hosting the sampled communities on both AWS, Kaggle, and Hugging Face. Please choose the resource that works best for you in the following.

In [ ]:
# Please set the following variable.
communities_repo="kaggle" #@param ["kaggle", "aws"]

In [ ]:
sampled_communities_dir=""
if communities_repo == "kaggle":
  print("Downloading from Kaggle ...")
  kaggle_dataset_path = kagglehub.dataset_download("aab/bitcoin-graph-sampled-communities/version/1")
  sampled_communities_dir = os.path.join(kaggle_dataset_path, "script_to_script_200k")

elif communities_repo == "aws":
  print("Downloading from AWS ...")
  with s3.get_object(Bucket=bucket, Key="v1/sampled_communities/script_to_script_200k.zip")["Body"] as file_object:
    zip_data = file_object.read()

    extract_dir=os.path.join(working_dir, "sampled_communities")
    with zipfile.ZipFile(io.BytesIO(zip_data)) as zip_file:
      zip_file.extractall(path=extract_dir)
    sampled_communities_dir=os.path.join(extract_dir, "script_to_script_200k")

print(f"Sampled communities downloaded to: {sampled_communities_dir}")

The unzipped directory contains a `raw` directory (for simple loading into `PyG`, as discussed later), which in turn contains a separate subdirectory for each community. See the following tree structure:

```shell
> tree
.
└── script_to_script_200k
    └── raw
        ├── 202509082048222123
        │   ├── BitcoinScriptNode.tsv
        │   └── metadata.tsv
        ├── 202509082048237682
        │   ├── BitcoinS2S.tsv
        │   ├── BitcoinScriptNode.tsv
        │   └── metadata.tsv
        ├── 202509082048444566
        ...
        └── metadata.tsv
```

As an example, we will explore the contents of the `202509082048237682` community.

In [ ]:
community_dir=os.path.join(sampled_communities_dir, "raw", "202509082048237682")
os.listdir(community_dir)

In [ ]:
s2s_df = pd.read_csv(os.path.join(community_dir, "BitcoinS2S.tsv"), sep="\t")
script_nodes_df = pd.read_csv(os.path.join(community_dir, "BitcoinScriptNode.tsv"), sep="\t")
g_metadata_df = pd.read_csv(os.path.join(community_dir, "metadata.tsv"), sep="\t")

In [ ]:
s2s_df.head()

In [ ]:
script_nodes_df.head()

In [ ]:
g_metadata_df.head()

The communities directory also contains a `metadata.tsv` file containing the metadata for all communities.

In [ ]:
metadata_df = pd.read_csv(
    os.path.join(sampled_communities_dir, "raw", "metadata.tsv"),
    sep="\t")

In [ ]:
metadata_df.head()

# The _Fruits_ 🍎 **Building Models**


In the previous section, we reviewed general-purpose sampled communities. While useful, we highly recommend importing the full graph and using our tools to sample your own. This allows you to tailor the data to your application's specific requirements, which is extremely important.

The Bitcoin graph enables you to model entity behavior within deep, contextual neighborhoods. This allows ML models to distinguish entities by their behavioral patterns, largely independent of external data and with high certainty. The deeper and more application-driven the sampling, the more accurate the entity characterization can be (e.g., assigning a trust score based on a wallet's trading patterns). See [this page](https://eba.b1aab.ai/docs/bitcoin/sampling/overview) for details.

However, for this _101_ tutorial, we will use the generic, pre-sampled communities as a showcase. Our goal is to demonstrate a model that learns node (i.e., Bitcoin script/address) embeddings from its neighborhood, which we then use to cluster the nodes.

To keep this tutorial simple and fast, we will load pre-trained weights that were generated using these generic communities. This lets us skip the training step and focus on evaluating the model's output. The complete training scripts and steps are discussed in detail on [this page](https://github.com/B1AAB/GraphStudio/tree/main/quickstart/script_classification).

In [ ]:
config = json.load(open("/tmp/b1aab/graphstudio/quickstart/script_classification/config.json"))
saves_root = config["saves_root"]
out_dim = config["out_dim"]
batch_size = config["batch_size"]
hidden_channels = config["hidden_channels"]  # note that this has to be divisible by HEADS

Downloading a trained model, so we do not run training.
If interested in re-training the model, you may use
[this notebook](https://github.com/B1AAB/GraphStudio/blob/main/quickstart/script_classification/train.ipynb).

In [ ]:
model_save_path = os.path.join(working_dir, "best_encoder.pth")
s3.download_file(bucket, "v1/g101/best_encoder.pth", model_save_path)

Next we pre-process the "raw" sampled communities into a
[PyG](https://github.com/pyg-team/pytorch_geometric) dataset.
You may refer to the
[data loader source code](https://github.com/B1AAB/GraphStudio/blob/main/quickstart/script_classification/script_classification/data_loader.py)
for details.


In [ ]:
dataset = BitcoinScriptsDataset(root=sampled_communities_dir)
EDGE_DIM = getattr(dataset, "num_edge_features", dataset[0].edge_attr.size(1))

In [ ]:
graph_id_to_idx_map = {data.graph_id: i for i, data in enumerate(dataset)}

In [ ]:
n = len(dataset)
n_train = int(0.7 * n)
n_val   = int(0.15 * n)
n_test  = n - n_train - n_val

gen = torch.Generator().manual_seed(SEED)
train_raw, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [n_train, n_val, n_test], generator=gen
)

In [ ]:
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
)

In [ ]:
encoder = GraphEncoder(
    in_channels=dataset.num_node_features,
    hidden_channels=hidden_channels,
    out_channels=out_dim,
    edge_dim=EDGE_DIM
).to(device)

In [ ]:
ckpt = torch.load(model_save_path, map_location=device)
encoder.load_state_dict(ckpt["model_state"])

embeddings_for_analysis = embed_roots(encoder, test_loader, device)

In [ ]:
optimal_k, inertias, silhouettes, k_range, best_silhouettes = find_optimal_clusters_root(embeddings_for_analysis, max_k=15, random_state=33)
print(f"Optimal number of clusters: {optimal_k}")

In [ ]:
plot_find_optimal_clusters_root(optimal_k, inertias, silhouettes, k_range)

In [ ]:
def visualize_embeddings_umap(graph_embeddings, cluster_labels, title="Graph Embedding Clusters (UMAP)", figsize=(5, 5)):
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42, n_jobs=1)
    embeddings_2d = reducer.fit_transform(graph_embeddings)

    plt.figure(figsize=figsize)

    scatter = sns.scatterplot(
        x=embeddings_2d[:, 0],
        y=embeddings_2d[:, 1],
        hue=cluster_labels,
        palette="tab20",
        legend="full",
        s=50,
        alpha=0.8
    )

    plt.legend(title="Cluster ID", loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)

    scatter.set_title(title)
    plt.xlabel("UMAP Component 1")
    plt.ylabel("UMAP Component 2")
    plt.show()

In [ ]:
k_means = KMeans(n_clusters=optimal_k, random_state=11, n_init=10)
final_clusters = k_means.fit_predict(embeddings_for_analysis)
final_score = silhouette_score(embeddings_for_analysis, final_clusters, metric="cosine")

print(f"Final Silhouette (cosine) with K={optimal_k}: {final_score:.4f}")

Next, we'll visualize the dataset by clustering the embeddings that were generated for the root node of each neighborhood.

In [ ]:
visualize_embeddings_umap(embeddings_for_analysis, final_clusters, title=f"Root Clusters (K={optimal_k})")

## Comparing Communities by Graph-Level Features

You may also characterize the sampled communities using their graph-level features.
These include statistics that summarize the entire sampled subgraph,
such as the average block height or BTC value calculated per hop from the community's root node.

However, it is crucial to understand a key distinction when using these for evaluation.

The model we trained here performs node-level representation learning.
It is designed to generate a feature vector (an embedding)
for the root node of each sampled community, based on its neighborhood.
In contrast, the statistics described above are graph-level
features that describe the entire community.

Therefore, clustering the root node embeddings and then evaluating
those clusters based on graph-level features is a methodological mismatch,
and can lead to misleading conclusions.

For such a comparison to be valid,
the model must be adapted for graph-level representation learning.
This typically involves adding a readout or pooling function
(e.g., global mean pooling) after the GNN layers
to aggregate all node embeddings into a single embedding for the entire graph
(or other methods).

In [ ]:
def format_block_height_axis(ax):
    formatter = FuncFormatter(lambda x, pos: f'{x / 1000:.0f}k')
    ax.xaxis.set_major_formatter(formatter)

In [ ]:
def prepare_stats_for_plotting(stats_list, graph_ids):
    plot_data = []
    for i, stats_dict in enumerate(stats_list):
        graph_id = graph_ids[i]
        for hop_level, hop_stats in stats_dict.items():
            hop_transition_key = f"hop{hop_level}->hop{hop_level+1}"

            if "Value_avg" in hop_stats:
                plot_data.append({
                    "graph_id": graph_id,
                    "hop_transition": hop_transition_key,
                    "avg_value": hop_stats["Value_avg"],
                    "metric_type": "Avg Value"
                })
            if "BlockHeight_avg" in hop_stats:
                plot_data.append({
                    "graph_id": graph_id,
                    "hop_transition": hop_transition_key,
                    "avg_block_height": hop_stats["BlockHeight_avg"],
                    "metric_type": "Avg BlockHeight"
                })
            if "OriginalInDegree_avg" in hop_stats:
                plot_data.append({
                    "graph_id": graph_id,
                    "hop_transition": hop_transition_key,
                    "avg_original_in_degree": hop_stats["OriginalInDegree_avg"],
                    "metric_type": "Avg Original In-Degree"
                })
            if "OriginalOutDegree_avg" in hop_stats:
                plot_data.append({
                    "graph_id": graph_id,
                    "hop_transition": hop_transition_key,
                    "avg_original_out_degree": hop_stats["OriginalOutDegree_avg"],
                    "metric_type": "Avg Original Out-Degree"
                })

    return pd.DataFrame(plot_data)

In [ ]:
def compare_graph_stats(stats_list, graph_ids):
    df = prepare_stats_for_plotting(stats_list, graph_ids)
    hop_order = sorted(df["hop_transition"].dropna().unique(), key=lambda x: int(x.split("->")[0][3:]))
    color_palette = sns.color_palette("viridis", len(graph_ids))

    fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=True)

    ax1 = axes[0]
    ax2 = axes[1]
    ax3 = axes[2]
    ax4 = axes[3]

    sns.barplot(data=df[df["metric_type"] == "Avg Value"],
                y="hop_transition", x="avg_value", hue="graph_id",
                ax=ax1, order=hop_order, orient="h", palette=color_palette)
    ax1.set_title("Avg. Tx Value by Hop")
    ax1.set_xlabel("Average Value (Log10 BTC)")
    ax1.set_ylabel("Hop Transition")
    ax1.set_xscale("log")
    ax1.get_legend().remove()

    sns.barplot(data=df[df["metric_type"] == "Avg BlockHeight"],
                y="hop_transition", x="avg_block_height", hue="graph_id",
                ax=ax2, order=hop_order, orient="h", palette=color_palette)
    ax2.set_title("Avg. Block Height by Hop")
    ax2.set_xlabel("Average Block Height")
    ax2.set_ylabel("")
    ax2.get_legend().remove()
    format_block_height_axis(ax2)

    sns.barplot(data=df[df["metric_type"] == "Avg Original In-Degree"],
                y="hop_transition", x="avg_original_in_degree", hue="graph_id",
                ax=ax3, order=hop_order, orient="h", palette=color_palette)
    ax3.set_title("Avg. Original In-Degree by Hop")
    ax3.set_xlabel("Average Original In-Degree")
    ax3.set_ylabel("")
    ax3.get_legend().remove()

    sns.barplot(data=df[df["metric_type"] == "Avg Original Out-Degree"],
                y="hop_transition", x="avg_original_out_degree", hue="graph_id",
                ax=ax4, order=hop_order, orient="h", palette=color_palette)
    ax4.set_title("Avg. Original Out-Degree by Hop")
    ax4.set_xlabel("Average Original Out-Degree")
    ax4.set_ylabel("")
    ax4.get_legend().remove()

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [ ]:
def get_graphs_from_cluster(target_cluster_id, all_cluster_labels, dataset):
    indices = np.where(all_cluster_labels == target_cluster_id)[0]

    if len(indices) == 0:
        print(f"No graphs found for cluster ID {target_cluster_id}.")
        return []

    graphs = [dataset[i] for i in indices]
    return graphs

In [ ]:
a_g_id = get_graphs_from_cluster(1, final_clusters, test_dataset)[0].graph_id
b_g_id = get_graphs_from_cluster(1, final_clusters, test_dataset)[1].graph_id
compare_graph_stats([dataset.per_graph_stats[a_g_id], dataset.per_graph_stats[b_g_id]], [a_g_id, b_g_id])

# Conclusion

The model evaluated here is for demonstration purposes only, as are the communities it was trained on. The goal of this **101** is to illustrate what the data looks like and how to work with it.


However, given the data's heterogeneity and scale, its 16-year temporal span (during which every aspect has evolved), the complex ETL pipeline involved, and the breadth and depth of potential applications, each use case demands a well-tailored, application-specific solution.


For instance, an application studying funds transferred between scripts 2 hops away from the Coinbase node (involving mostly miners) will differ significantly from one studying communities at least 5 hops away (representing mostly user trades).


Therefore, we suggest you consider the detailed statistical profile of the blockchain (provided in the manuscript), then design your own sampling and model architecture that best matches your application.


On [this page](https://github.com/B1AAB/GraphStudio?tab=readme-ov-file#designing--evaluating-models-on-the-bitcoin-graph),
we provide general recommendations for designing applications with this dataset.

# Your Turn: Take the Next Step!

You've seen the basics, but this dataset's real power is in tackling large-scale, real-world challenges. Here are a few high-impact directions you can explore next:

### Build On-Chain Reputation Systems

Bitcoin is pseudonymous, but behavior isn't. Can you build a GNN that learns an entity's trading patterns (from its deep, temporal neighborhood) to assign it a **"trust score"**?

Most current methods are either not scalable or rely on hand-tuned rules. A robust, ML-first score could be revolutionary for Decentralized Finance (DeFi) applications, such as enabling under-collateralized lending. You could even use it to power a real-time _"Internet Security"_ app for crypto, **flagging transactions to prevent fraud before they are confirmed.**

### Stress-Test Your GNN Architecture

This graph is a real-world benchmark for GNNs. With over 2.4 billion nodes and ~40 billion edges, it can stress-test your model's scalability at a level you'd normally only see in production---and what better production dataset is there than 16 years of Bitcoin trades?! 😀

More importantly, its 16-year history contains massive, **real-world temporal distribution shifts.** Can your model generalize, or will a model trained on 2017's data fail on 2025's? This is the perfect dataset to assess your model's robustness and scalability.


### Augment with Off-Chain Data

We've focused on-chain, but the real magic happens when you combine it with other data. On-chain data gives you the **"what"** (the raw facts of a transaction), while off-chain data provides the **"why"** (the context, e.g., _"Was this payment for gambling?"_). Combining these two worlds is key to boosting the performance of your models.

You can augment the graph with market indicators (like OHLC prices) to build more powerful forecasting models. You could also correlate on-chain activity with real-world events (like policy announcements or social media sentiment) to quantify their impact on the ecosystem. We provide an example of using off-chain data on [this page](https://github.com/B1AAB/GraphStudio/tree/main/off_chain_resources), but countless other resources are waiting to be added!